In [1]:
import chess.pgn
import pandas as pd
import chess
import numpy as np
from scipy import linalg

In [2]:
ls -shr

total 90M
 16K README.md                              572K chess.ipynb
 89M lichess_db_standard_rated_2013-01.pgn   52K chess-and-eigenvectors.ipynb


In [3]:
board = chess.Board()

In [4]:
pgn = open('lichess_db_standard_rated_2013-01.pgn',encoding="utf-8")

In [5]:
#creates a list with one entry for each space on the board
spaces = []
for i in range(1,9):
    for j in range (1,9):
        spaces.append(chr(j+96)+str(i))

In [6]:
#create a matrix to tally weights for every pair of opposing pieces
pieces_list = [space for space in spaces if any(map(space.__contains__, ['1','2','7','8']))]
weights = pd.DataFrame(0,index=pieces_list, columns=pieces_list)

In [7]:
#create a state dictionary of each square and the initial square of each piece
#Those 'None' entries are the empty squares in the middle of the board, before the first move
#Each piece is on it's own square to begin
state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}
state_dict

{'a1': 'a1',
 'b1': 'b1',
 'c1': 'c1',
 'd1': 'd1',
 'e1': 'e1',
 'f1': 'f1',
 'g1': 'g1',
 'h1': 'h1',
 'a2': 'a2',
 'b2': 'b2',
 'c2': 'c2',
 'd2': 'd2',
 'e2': 'e2',
 'f2': 'f2',
 'g2': 'g2',
 'h2': 'h2',
 'a3': None,
 'b3': None,
 'c3': None,
 'd3': None,
 'e3': None,
 'f3': None,
 'g3': None,
 'h3': None,
 'a4': None,
 'b4': None,
 'c4': None,
 'd4': None,
 'e4': None,
 'f4': None,
 'g4': None,
 'h4': None,
 'a5': None,
 'b5': None,
 'c5': None,
 'd5': None,
 'e5': None,
 'f5': None,
 'g5': None,
 'h5': None,
 'a6': None,
 'b6': None,
 'c6': None,
 'd6': None,
 'e6': None,
 'f6': None,
 'g6': None,
 'h6': None,
 'a7': 'a7',
 'b7': 'b7',
 'c7': 'c7',
 'd7': 'd7',
 'e7': 'e7',
 'f7': 'f7',
 'g7': 'g7',
 'h7': 'h7',
 'a8': 'a8',
 'b8': 'b8',
 'c8': 'c8',
 'd8': 'd8',
 'e8': 'e8',
 'f8': 'f8',
 'g8': 'g8',
 'h8': 'h8'}

In [ ]:
game_counter = 0
total_move_counter = 0
game = chess.pgn.read_game(pgn)

while game is not None:
    counter = 0
    state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}

    #print(game)
    #do stuff here



    #for each line in the file
    for move in game.mainline_moves():
        counter += 1
        if board.is_pseudo_legal(move):


            if board.is_capture(move):
                capturing_piece = state_dict[chess.square_name(move.from_square)]
                captured_piece = state_dict[chess.square_name(move.to_square)]
                #print('capturing piece',capturing_piece)
                #print('captured piece',captured_piece)

                if board.is_en_passant(move):
                    #do en passant stuff here
                    #print("en passant")
                    pass
                elif capturing_piece is not None:
                    if captured_piece is not None:

                        #record the kill in a matrix of all kills
                        #print(counter,chess.square_name(move.from_square), "takes on",chess.square_name(move.to_square))
                        #print(counter,state_dict[chess.square_name(move.from_square)], "takes",state_dict[chess.square_name(move.to_square)])

                        #what piece is there now?
                     

                        #increment an entry in the weights matrix when one piece takes another
                        weights.loc[state_dict[chess.square_name(move.from_square)],state_dict[chess.square_name(move.to_square)]] += 1

                        #update the piece on the target square
                        state_dict[chess.square_name(move.to_square)] = state_dict[chess.square_name(move.from_square)]

                        #update the source square to be empty
                        state_dict[chess.square_name(move.from_square)] = None

                        #board.push(move)  # Make the move


                        #if df.loc[i+1,'move_no'] == 1:
                        #    state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}

            else:


                #add castling special cases to record both moves
                if board.is_kingside_castling(move):
                    if board.turn:
                        #print(counter,'... and white castles kingside')
                        #move the appopriate rook
                        state_dict['f1'] = 'h1'
                        #leaving the rook source square empty
                        state_dict['h1'] = 'None'
                        #board.push(move)  # Make the move



                    elif not board.turn:
                        #print(counter,'... and black castles kingside')
                        #move the appopriate rook
                        state_dict['f8'] = 'h8'
                        #leaving the rook source square empty
                        state_dict['h8'] = 'None'
                        #board.push(move)  # Make the move

                elif board.is_queenside_castling(move):
                    if board.turn:
                        #print(counter,'... and white castles queenside')
                        #move the appopriate rook
                        state_dict['d1'] = 'a1'
                        #leaving the rook source square empty
                        state_dict['a1'] = 'None'
                        #board.push(move)  # Make the move

                    elif not board.turn:
                        #print(counter,'... and black castles queenside')
                        #move the appopriate rook
                        state_dict['d8'] = 'a8'
                        #leaving the rook source square empty
                        state_dict['a8'] = 'None'
                        #board.push(move)  # Make the move

                else:
                    #update the piece on the target square
                    state_dict[chess.square_name(move.to_square)] = state_dict[chess.square_name(move.from_square)]

                    #update the source square to be empty
                    state_dict[chess.square_name(move.from_square)] = None


                    #print(counter, chess.square_name(move.from_square),chess.square_name(move.to_square))

        board.push(move)  # Make the move
        #print(state_dict)
            
    if board.is_checkmate():
        checkers_list = [state_dict[chess.square_name(x)] for x in list(board.checkers())]
        if checkers_list[0] is not None:
            piece_giving_check = str(checkers_list[0])

            if board.turn:
                captured_king = 'e1'
            else:
                captured_king = 'e8'
            weights.loc[piece_giving_check,captured_king] += 1

    
    game_counter += 1
    #print('end of game',game_counter)
    
    total_move_counter += counter
    #print('total moves analyzed',total_move_counter)

    
    #print('board outcome',board.outcome())


    #print('is game over?',board.is_game_over())
    board.reset()

    game = chess.pgn.read_game(pgn)




    #print(state_dict)

    

In [ ]:
print(game_counter,'games analyzed')
print(total_move_counter,'moves analyzed')

In [ ]:
weights

In [ ]:
#subtracting the identity matrix is necessary, forming an implicit set of homogeneous equations
a = weights - np.identity(32)
#print(a)

In [ ]:
b = np.zeros(32)
#print(b)

In [ ]:
x = linalg.solve(a, b)
x

### Why doesn't it work?
### Because the solver is finding the trivial solution. i.e. just a vector full of zeroes.
### To check that there is actually a non trivial solution, the determinant of A must be non-zero.

In [ ]:
np.linalg.det(a)

### OK since the determinant is not zero, there must be at least one non-trivial solution. How to find it?
### An homogenous linear equation can be solved by finding the eigenvectors.

In [ ]:
eigen_values, eigen_vectors = np.linalg.eig(a)  
eigen_vectors

In [ ]:
values = eigen_vectors[:, np.argmax(eigen_values)] 
values
#Notice that the entries of the eigenvector corresponding to the largest eigenvalue are all real valued,
#So it's OK to throw away the imaginary component

In [ ]:
## Let's normalize the values


In [ ]:
list = []
for i in values:
    list.append(abs(float(i)))

In [ ]:
#The fractional inverse of the average of all the pawns
factor = 1 / np.mean(list[8:24])

In [ ]:
normed = [i*factor for i in list]
#normed

In [ ]:
normalized_list = ["%.2f"%i for i in normed]

In [ ]:
print(normalized_list[24:32])
print(normalized_list[16:24])
print(normalized_list[8:16])
print(normalized_list[0:8])


In [ ]:
#tempo difference
white_sum = sum(normed[0:16])
black_sum = sum(normed[16:32])


print('total value of white pieces is', white_sum)
print('total value of black pieces is', black_sum)
print('total difference in value between white and black pieces is', white_sum-black_sum)
#This implies that white's tempo advantage is worth about 3 thousandths of an average pawn.

In [ ]:
#Notice that, for each pair of opposing pawns, the black pawn is stronger in 7 out of 8 pairs
#Why should this be? Possibly because white has the initial tempo for attacking early in the game,
#giving the black pawns many opportunities for capturing pieces defensively, and practically necessitating it.

In [ ]:
#find average strength of all the pieces taken by each piece
#eg, rook strength seems low in general, probably because they come out later in the game
#and have fewer opportunities to capture lots of pieces,
#but they probably capture, on average, stronger pieces, in proportion to their own strength 

In [ ]:
#Try a Matrix Decomposition

In [ ]:
U, D, V = np.linalg.svd(a, full_matrices=True)

In [ ]:
D